<a href="https://colab.research.google.com/github/hareshbaskaran/Sarcasm-on-Reddit/blob/master/sarcasm_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [5]:
train_df=pd.read_csv('/content/train-balanced-sarcasm.csv')
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2.0,-1.0,-1.0,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4.0,-1.0,-1.0,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3.0,3.0,0.0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8.0,-1.0,-1.0,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6.0,-1.0,-1.0,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [6]:
len(train_df),train_df.index.shape[-1]


(37390, 37390)

In [7]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go


In [8]:
print("Sntiment - ".format(),train_df.shape)

Sntiment -  (37390, 10)


In [9]:
df = train_df[["comment"]]
df["comment"] = df["comment"].astype(str)
train_df.head()

<ipython-input-9-f87a1dc94e64>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2.0,-1.0,-1.0,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4.0,-1.0,-1.0,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3.0,3.0,0.0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8.0,-1.0,-1.0,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6.0,-1.0,-1.0,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [10]:
#lowercasing 
df["text_lower"] = df["comment"].str.lower()
df.head()

<ipython-input-10-2c052aac36c1>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower
0,NC and NH.,nc and nh.
1,You do know west teams play against west teams...,you do know west teams play against west teams...
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g..."
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni..."
4,I could use one of those tools.,i could use one of those tools.


In [11]:
#punctuation removal
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["comment"].apply(lambda text: remove_punctuation(text))
df.head()

<ipython-input-11-0bcf7df8cb62>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower,text_wo_punct
0,NC and NH.,nc and nh.,NC and NH
1,You do know west teams play against west teams...,you do know west teams play against west teams...,You do know west teams play against west teams...
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...",They were underdogs earlier today but since Gr...
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...",This meme isnt funny none of the new york nigg...
4,I could use one of those tools.,i could use one of those tools.,I could use one of those tools


In [12]:
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [13]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

<ipython-input-13-b6fef9350b19>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower,text_wo_punct,text_wo_stop
0,NC and NH.,nc and nh.,NC and NH,NC NH
1,You do know west teams play against west teams...,you do know west teams play against west teams...,You do know west teams play against west teams...,You know west teams play west teams east teams...
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...",They were underdogs earlier today but since Gr...,They underdogs earlier today since Gronks anno...
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...",This meme isnt funny none of the new york nigg...,This meme isnt funny none new york nigga ones
4,I could use one of those tools.,i could use one of those tools.,I could use one of those tools,I could use one tools


In [14]:
#removal of frequent words
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(20)

[('I', 6296),
 ('like', 1933),
 ('dont', 1284),
 ('would', 1215),
 ('get', 1182),
 ('people', 1116),
 ('one', 1092),
 ('Im', 1080),
 ('You', 1021),
 ('Yeah', 975),
 ('The', 946),
 ('think', 903),
 ('know', 858),
 ('But', 841),
 ('good', 730),
 ('Its', 689),
 ('right', 686),
 ('really', 667),
 ('time', 629),
 ('thats', 628)]

In [15]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

<ipython-input-15-0ba26e427506>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,NC and NH.,nc and nh.,NC and NH,NC NH,NC NH
1,You do know west teams play against west teams...,you do know west teams play against west teams...,You do know west teams play against west teams...,You know west teams play west teams east teams...,know west teams play west teams east teams right
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...",They were underdogs earlier today but since Gr...,They underdogs earlier today since Gronks anno...,They underdogs earlier today since Gronks anno...
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...",This meme isnt funny none of the new york nigg...,This meme isnt funny none new york nigga ones,This meme isnt funny none new york nigga ones
4,I could use one of those tools.,i could use one of those tools.,I could use one of those tools,I could use one tools,could use tools


In [16]:
#Reduction of inflected words Methods
#STEMMING
from nltk.stem.porter import PorterStemmer

# Drop the two columns 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["text_stemmed"] = df["text_wo_stopfreq"].apply(lambda text: stem_words(text))
df.head()


<ipython-input-16-7f1e8e9df582>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_stemmed
0,NC and NH.,nc and nh.,NC and NH,NC NH,NC NH,nc nh
1,You do know west teams play against west teams...,you do know west teams play against west teams...,You do know west teams play against west teams...,You know west teams play west teams east teams...,know west teams play west teams east teams right,know west team play west team east team right
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...",They were underdogs earlier today but since Gr...,They underdogs earlier today since Gronks anno...,They underdogs earlier today since Gronks anno...,they underdog earlier today sinc gronk announc...
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...",This meme isnt funny none of the new york nigg...,This meme isnt funny none new york nigga ones,This meme isnt funny none new york nigga ones,thi meme isnt funni none new york nigga one
4,I could use one of those tools.,i could use one of those tools.,I could use one of those tools,I could use one tools,could use tools,could use tool


In [17]:
#Snowball stemmer 
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [18]:
#lemmatization
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text_stemmed"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-18-c1e0dbd618d4>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_stemmed,text_lemmatized
0,NC and NH.,nc and nh.,NC and NH,NC NH,NC NH,nc nh,nc nh
1,You do know west teams play against west teams...,you do know west teams play against west teams...,You do know west teams play against west teams...,You know west teams play west teams east teams...,know west teams play west teams east teams right,know west team play west team east team right,know west team play west team east team right
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...",They were underdogs earlier today but since Gr...,They underdogs earlier today since Gronks anno...,They underdogs earlier today since Gronks anno...,they underdog earlier today sinc gronk announc...,they underdog earlier today sinc gronk announc...
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...",This meme isnt funny none of the new york nigg...,This meme isnt funny none new york nigga ones,This meme isnt funny none new york nigga ones,thi meme isnt funni none new york nigga one,thi meme isnt funni none new york nigga one
4,I could use one of those tools.,i could use one of those tools.,I could use one of those tools,I could use one tools,could use tools,could use tool,could use tool


In [19]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text_stemmed"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
<ipython-input-19-c020611f0b12>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,comment,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_stemmed,text_lemmatized
0,NC and NH.,nc and nh.,NC and NH,NC NH,NC NH,nc nh,nc nh
1,You do know west teams play against west teams...,you do know west teams play against west teams...,You do know west teams play against west teams...,You know west teams play west teams east teams...,know west teams play west teams east teams right,know west team play west team east team right,know west team play west team east team right
2,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...",They were underdogs earlier today but since Gr...,They underdogs earlier today since Gronks anno...,They underdogs earlier today since Gronks anno...,they underdog earlier today sinc gronk announc...,they underdog early today sinc gronk announc a...
3,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...",This meme isnt funny none of the new york nigg...,This meme isnt funny none new york nigga ones,This meme isnt funny none new york nigga ones,thi meme isnt funni none new york nigga one,thi meme isnt funni none new york nigga one
4,I could use one of those tools.,i could use one of those tools.,I could use one of those tools,I could use one tools,could use tools,could use tool,could use tool
